In [1]:
import os

import numpy as np
from models.attention import multihead_model
from models.selfattention import selfattention_model, selfattention_model_modified
from models.baseline_model import baseline, baseline_residual
from keras.callbacks import ModelCheckpoint, EarlyStopping
from data_utility.file_utility import FileUtility
from data_utility.labeling_utility import LabelingData
import itertools
from data_utility.feedgenerator import train_batch_generator_408, validation_batch_generator_408


os.environ["CUDA_VISIBLE_DEVICES"] = '1'

Using TensorFlow backend.


In [2]:
def run_transformer_attention(epochs=10, setting_name='attention', gpu='0', train_batch_size=64,
                 test_batch_size=100, patience=10, features_to_use=['onehot', 'sequence_profile'], convs=[3, 5, 7], dense_size=200, lstm_size=400,use_CRF=False):
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu

    # read files
    train_file = '../DeepSeq2Sec/data/s8_all_features/train.txt'
    test_file = '../DeepSeq2Sec/data/s8_all_features/test.txt'

    LD = LabelingData(train_file, test_file)
    train_lengths = [int(j) for j in FileUtility.load_list('/'.join(train_file.split('/')[0:-1]) + '/train_length.txt')]
    test_lengths = [int(i) for i in FileUtility.load_list('/'.join(test_file.split('/')[0:-1]) + '/test_length.txt')]

    # model
    model, params = multihead_model(LD.n_classes, features_to_use=features_to_use, convs=convs,
                             dense_size=dense_size, lstm_size=lstm_size,use_CRF=use_CRF, filter_size=256)

    # output directory
    FileUtility.ensure_dir('results/' + setting_name + params + '/')

    # save model
    with open('results/' + setting_name + params + '/' + 'config.txt', 'w') as fh:
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    # check points
    filepath = 'results/' + setting_name + params + "/weights-improvement-{epoch:02d}-{weighted_acc:.3f}-{val_weighted_acc:.3f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_weighted_acc', verbose=1, save_best_only=True, mode='max',
                                 period=1)
    earlystopping = EarlyStopping(monitor='val_weighted_acc', min_delta=0, patience=patience, verbose=0, mode='max',
                                  baseline=None)
    callbacks_list = [checkpoint, earlystopping]

    # calculate the sizes
    steps_per_epoch = len(train_lengths) / train_batch_size if len(train_lengths) % train_batch_size == 0 else int(
        len(train_lengths) / train_batch_size) + 1
    validation_steps = int(len(test_lengths) / test_batch_size) if len(test_lengths) % test_batch_size == 0 else int(
        len(test_lengths) / test_batch_size) + 1

    # feed model
    h = model.fit_generator(train_batch_generator_408(train_batch_size), steps_per_epoch=steps_per_epoch,
                            validation_data=validation_batch_generator_408(test_batch_size),
                            validation_steps=validation_steps,
                            shuffle=False, epochs=epochs, verbose=1, callbacks=callbacks_list)

    # save the history
    FileUtility.save_obj('results/' + setting_name + params + '/history', h.history)





In [3]:
run_transformer_attention(epochs=300, setting_name='multihead_lstm__cnn_time', gpu='1', train_batch_size=64,
                 test_batch_size=100, patience=15, features_to_use=['onehot', 'sequence_profile'], convs=[3,5,7,11,21], dense_size=1000, lstm_size=1000,use_CRF=False)

Labeling utility object created..
Training y encoded shape is  (5534, 700)
Maximum sequence length is 700
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 408)    0                                            
__________________________________________________________________________________________________
onehot (Lambda)                 (None, None, 21)     0           input_1[0][0]                    
__________________________________________________________________________________________________
sequenceprofile (Lambda)        (None, None, 21)     0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 42)     0           onehot[0][0]                     
   

In [3]:
def run_selfattention_modified(epochs=10, setting_name='attention', gpu='0', train_batch_size=64,
                 test_batch_size=100, patience=10, features_to_use=['onehot', 'sequence_profile'], convs=[3, 5, 7], dense_size=200, lstm_size=400,use_CRF=False):
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu

    # read files
    train_file = '../DeepSeq2Sec/data/s8_all_features/train.txt'
    test_file = '../DeepSeq2Sec/data/s8_all_features/test.txt'

    LD = LabelingData(train_file, test_file)
    train_lengths = [int(j) for j in FileUtility.load_list('/'.join(train_file.split('/')[0:-1]) + '/train_length.txt')]
    test_lengths = [int(i) for i in FileUtility.load_list('/'.join(test_file.split('/')[0:-1]) + '/test_length.txt')]

    # model
    model, params = selfattention_model_modified(LD.n_classes, features_to_use=features_to_use, convs=convs,
                             dense_size=dense_size, lstm_size=lstm_size,use_CRF=use_CRF, filter_size=256)

    # output directory
    FileUtility.ensure_dir('results/' + setting_name + params + '/')

    # save model
    with open('results/' + setting_name + params + '/' + 'config.txt', 'w') as fh:
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    # check points
    filepath = 'results/' + setting_name + params + "/weights-improvement-{epoch:02d}-{weighted_acc:.3f}-{val_weighted_acc:.3f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_weighted_acc', verbose=1, save_best_only=True, mode='max',
                                 period=1)
    earlystopping = EarlyStopping(monitor='val_weighted_acc', min_delta=0, patience=patience, verbose=0, mode='max',
                                  baseline=None)
    callbacks_list = [checkpoint, earlystopping]

    # calculate the sizes
    steps_per_epoch = len(train_lengths) / train_batch_size if len(train_lengths) % train_batch_size == 0 else int(
        len(train_lengths) / train_batch_size) + 1
    validation_steps = int(len(test_lengths) / test_batch_size) if len(test_lengths) % test_batch_size == 0 else int(
        len(test_lengths) / test_batch_size) + 1

    # feed model
    h = model.fit_generator(train_batch_generator_408(train_batch_size), steps_per_epoch=steps_per_epoch,
                            validation_data=validation_batch_generator_408(test_batch_size),
                            validation_steps=validation_steps,
                            shuffle=False, epochs=epochs, verbose=1, callbacks=callbacks_list)

    # save the history
    FileUtility.save_obj('results/' + setting_name + params + '/history', h.history)
    return model




In [4]:
model=run_selfattention_modified(epochs=300, setting_name='self-attention_withconv_2drops_dense_instance_norm_new_', gpu='1', train_batch_size=64,
                 test_batch_size=100, patience=15, features_to_use=['onehot', 'sequence_profile'], convs=[3,5,7,11,21], dense_size=1000, lstm_size=1000,use_CRF=False)

Labeling utility object created..
Training y encoded shape is  (5534, 700)
Maximum sequence length is 700
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 408)    0                                            
__________________________________________________________________________________________________
onehot (Lambda)                 (None, None, 21)     0           input_1[0][0]                    
__________________________________________________________________________________________________
sequenceprofile (Lambda)        (None, None, 21)     0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 42)     0           onehot[0][0]                     
   

Epoch 7/300
87/87 [==============================] - 27s 310ms/step - loss: 0.8191 - weighted_acc: 0.7298 - val_loss: 0.9849 - val_weighted_acc: 0.6752

Epoch 00007: val_weighted_acc improved from 0.67328 to 0.67522, saving model to results/self-attention_withconv_2drops_dense_instance_norm_new_selfattention#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_256/weights-improvement-07-0.730-0.675.hdf5
Epoch 8/300
87/87 [==============================] - 27s 309ms/step - loss: 0.8111 - weighted_acc: 0.7328 - val_loss: 0.9719 - val_weighted_acc: 0.6790

Epoch 00008: val_weighted_acc improved from 0.67522 to 0.67902, saving model to results/self-attention_withconv_2drops_dense_instance_norm_new_selfattention#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_256/weights-improvement-08-0.733-0.679.hdf5
Epoch 9/300
87/87 [==============================] - 26s 304ms/step - loss: 0.7988 - weighted_acc: 0.7356 - val_loss: 0.9738 -